# 識別対象のサンプルデータ
架空のデータを使って２項識別器の例を見ていきます。

ここで使うデータは架空のデータですが、ベースは[都道府県別統計とランキングで見る県民性-東西対立型ランキング](http://todo-ran.com/t/type/1) から持ってきたデータを加工して作ったものです。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
with open('data.csv', encoding="utf-8") as f:
    print(f.read())

In [ ]:
data = pd.read_csv("data.csv", delimiter=",", encoding="utf-8")  # header がない場合は header=None

In [ ]:
X=data[['平均気温','納豆消費量']]
y=data['東西ラベル']

In [ ]:
X.head(8)  # 上から8個表示 

# 元のデータのプロット


In [ ]:
plt.scatter(X['平均気温'],X['納豆消費量'],c=y,s=50)
plt.show()

# 標準化と正規化
平均気温と納豆消費量では分布の幅が違いすぎる。識別器によっては両者の粒度を合わせないとまったくうまくいかない。

In [ ]:
# 標準化
sc = StandardScaler()
data_std = sc.fit_transform(X)
 
# 正規化
ms = MinMaxScaler()
data_norm = ms.fit_transform(X)

# 標準化
平均が０，標準偏差が１となるように線形変換を施す。

In [ ]:
# matplotlibの準備
import matplotlib.pyplot as plt
%matplotlib inline
plt.set_cmap(plt.cm.Paired) # 色設定

plt.scatter(data_std[:,0],data_std[:,1],c=y,s=50)
plt.show()

# 正規化
データが０〜１の範囲になるように線形変換をほどこす。

In [ ]:
plt.scatter(data_norm[:,0],data_norm[:,1],c=y,s=50)
plt.show()

In [ ]:
# 境界線を引く関数の定義

def plotBoundary(X, clf, mesh=True, boundary=True, n_neighbors=1):

    # plot range
    x_min = min(X[:,0])
    x_max = max(X[:,0])
    y_min = min(X[:,1])
    y_max = max(X[:,1])

    # visualizing decision function
    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j] # make a grid
    
    Z = clf.predict(np.c_[XX.ravel(), YY.ravel()]) # evaluate the value 
    
    Z = Z.reshape(XX.shape) # just reshape

    if mesh:
        plt.pcolormesh(XX, YY, Z, zorder=-10) # paint in 2 colors, if Z > 0 or not

    if boundary:
        plt.contour(XX, YY, Z, 
                    colors='k', linestyles='-', levels=[0])

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

# 最近傍識別(k-nn で n=1)
未知のデータの識別結果＝既知のデータで、最も近いデータと同じ結果

In [ ]:
from sklearn import neighbors

# 識別オブジェクト作成
clf = neighbors.KNeighborsClassifier(n_neighbors=1)

In [ ]:
clf.fit(data_std, y) # 　　標準化データを用いて学習

plt.scatter(data_std[:, 0], data_std[:,1], marker='o', s=50, c=y)

plotBoundary(data_std, clf) # 境界線の描画

In [ ]:
clf.fit(data_norm, y) # 正規化データを用いて学習

plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)

plotBoundary(data_norm, clf) # 境界線の描画

# k-最近傍識別
k=3 、すなわち、最も近い既知データを３つ選びそれらの多数決で判定結果を決定

In [ ]:
# 識別オブジェクト作成
clf = neighbors.KNeighborsClassifier(n_neighbors=3)
clf.fit(data_std, y) # 正規化データを用いて学習
plt.scatter(data_std[:, 0], data_std[:,1], marker='o', s=50, c=y)
plotBoundary(data_std, clf) # 境界線の描画

In [ ]:
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

# ロジスティック回帰
単純パーセプトロンで特性関数をロジスティック関数に設定したもの

In [ ]:
# モジュールの準備
from sklearn import linear_model
# オブジェクト作成
clf = linear_model.LogisticRegression()
clf.fit(data_std, y) # 正規化データを用いて学習
plt.scatter(data_std[:, 0], data_std[:,1], marker='o', s=50, c=y)
plotBoundary(data_std, clf) # 境界線の描画

In [ ]:
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

ロジスティック回帰は見ての通り、線型識別である。

# 線型SVM （サポートベクターマシン）

データをまっすぐなひとつの識別面（線）だけで分離するという点ではロジスティック回帰と同じだが、識別面と両クラスのデータのマージンが最大になるように識別面を最適化する。
<img src ="https://camo.qiitausercontent.com/3b09b1e5b79149434df411160f07ccfde4dd70eb/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f33343732312f38333530323330312d346435302d386165612d643266322d3339633232613933616166392e706e67" width=300>
[線型SVM](https://qiita.com/rennnosuke/items/cd01aa855196340167df)


In [ ]:
# モジュールの準備
from sklearn import svm
# オブジェクト作成
clf = svm.LinearSVC(C=1)
clf.fit(data_std, y) # 正規化データを用いて学習
plt.scatter(data_std[:, 0], data_std[:,1], marker='o', s=50, c=y)
plotBoundary(data_std, clf) # 境界線の描画

In [ ]:
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

上の図で、識別面の両側にある色の違う点は、そのようなデータは誤認識してしまうことを意味する。

これは SVMのアルゴリズムがある程度の誤認識の許容度の範囲でデータ全体でのマージンを最大化するというアルゴリズムだからである。

<img src ="https://goo.gl/i7skbi" width=300>

scikit-learn の 線型SVMはパラメータ設定で許容度を変更することもできる。

> clf = svm.LinearSVC(C=1)

Cはエラーに対するペナルティの大きさを意味し、デフォルトは１である。ためしに、C=100でやってみよう。

In [ ]:
clf = svm.LinearSVC(C=100)
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

# 非線形SVM

なるべく多くのマージンを確保するという基本的なアイデアはそのままに、識別面として曲面を許すように拡張した SVM

> clf = svm.SVC(kernel='rbf', C=1.)

kernel というパラメータで、識別面の種類を指定する。

- 'linear’  平面（LinearSVCとおなじになる）
- ‘poly’ 多項式面
- ‘rbf’ ガウシアン関数　　sklearn のデフォルト
- ‘sigmoid’　シグモイド関数

参考

- [Support Vector Machines](http://scikit-learn.org/stable/modules/svm.html)
- [非線形SVM](https://qiita.com/rennnosuke/items/fab837825b64bf50be56)


In [ ]:
# モジュールの準備
from sklearn import svm
# オブジェクト作成
clf = svm.SVC(kernel='rbf')

In [ ]:
clf.fit(data_std, y) # 正規化データを用いて学習
plt.scatter(data_std[:, 0], data_std[:,1], marker='o', s=50, c=y)
plotBoundary(data_std, clf) # 境界線の描画

In [ ]:
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

はみ出るデータが多いのでペナルティ項を大きくしてみる

In [ ]:
clf = svm.SVC(kernel='rbf', C=10)
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

In [ ]:
clf = svm.SVC(kernel='rbf', C=30)
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画

In [ ]:
clf = svm.SVC(kernel='rbf', C=35)
clf.fit(data_norm, y) # 正規化データを用いて学習
plt.scatter(data_norm[:, 0], data_norm[:,1], marker='o', s=50, c=y)
plotBoundary(data_norm, clf) # 境界線の描画